In [1]:
import os
from dotenv import load_dotenv

load_dotenv()

os.environ["LANGCHAIN_API_KEY"]= os.getenv("LANGCHAIN_API_KEY")
os.environ["LANGCHAIN_TRACING_V2"]="true"
os.environ["LANGCHAIN_PROJECT"]=os.getenv("LANGCHAIN_PROJECT")

In [2]:
from langchain_community.document_loaders import WebBaseLoader


USER_AGENT environment variable not set, consider setting it to identify your requests.


In [3]:
loader = WebBaseLoader("https://docs.langchain.com/langsmith/server-api-ref#agent-server-api-reference-for-langsmith-deployment")

In [4]:
docs = loader.load()
docs

[Document(metadata={'source': 'https://docs.langchain.com/langsmith/server-api-ref#agent-server-api-reference-for-langsmith-deployment', 'title': 'Agent Server API reference for LangSmith Deployment - Docs by LangChain', 'language': 'en'}, page_content='Agent Server API reference for LangSmith Deployment - Docs by LangChainSkip to main contentDocs by LangChain home pageLangSmithSearch...⌘KSupportGitHubTry LangSmithTry LangSmithSearch...NavigationAgent Server APIAgent Server API reference for LangSmith DeploymentGet startedObservabilityEvaluationPrompt engineeringDeploymentPlatform setupReferenceOverviewLangSmith Python SDKLangSmith JS/TS SDKLangGraph Python SDKLangGraph JS/TS SDKLangSmith APILangSmith DeploymentAgent Server APIOverviewAssistantsThreadsThread RunsCrons (Plus tier)Stateless RunsStoreA2AMCPSystemControl Plane APILangGraph CLIRemoteGraphAgent Server environment variablesReleasesAgent Server changelogSelf-hosted changelogRelease versionsOn this pageAuthenticationLangSmith D

In [5]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200
)

documents= text_splitter.split_documents(docs)

In [6]:
documents

[Document(metadata={'source': 'https://docs.langchain.com/langsmith/server-api-ref#agent-server-api-reference-for-langsmith-deployment', 'title': 'Agent Server API reference for LangSmith Deployment - Docs by LangChain', 'language': 'en'}, page_content='Agent Server API reference for LangSmith Deployment - Docs by LangChainSkip to main contentDocs by LangChain home pageLangSmithSearch...⌘KSupportGitHubTry LangSmithTry LangSmithSearch...NavigationAgent Server APIAgent Server API reference for LangSmith DeploymentGet startedObservabilityEvaluationPrompt engineeringDeploymentPlatform setupReferenceOverviewLangSmith Python SDKLangSmith JS/TS SDKLangGraph Python SDKLangGraph JS/TS SDKLangSmith APILangSmith DeploymentAgent Server APIOverviewAssistantsThreadsThread RunsCrons (Plus tier)Stateless RunsStoreA2AMCPSystemControl Plane APILangGraph CLIRemoteGraphAgent Server environment variablesReleasesAgent Server changelogSelf-hosted changelogRelease versionsOn this pageAuthenticationLangSmith D

In [ ]:
from langchain_community.embeddings import OllamaEmbeddings

embeddings = OllamaEmbeddings(model="nomic-embed-text:latest")


In [10]:
from langchain_community.vectorstores import FAISS

vectorstoredb = FAISS.from_documents(documents, embeddings)


In [11]:
vectorstoredb

In [12]:
query="For deployments to LangSmith, authentication is required"
result = vectorstoredb.similarity_search(query)
result[0].page_content

'Agent Server API reference for LangSmith Deployment - Docs by LangChainSkip to main contentDocs by LangChain home pageLangSmithSearch...⌘KSupportGitHubTry LangSmithTry LangSmithSearch...NavigationAgent Server APIAgent Server API reference for LangSmith DeploymentGet startedObservabilityEvaluationPrompt engineeringDeploymentPlatform setupReferenceOverviewLangSmith Python SDKLangSmith JS/TS SDKLangGraph Python SDKLangGraph JS/TS SDKLangSmith APILangSmith DeploymentAgent Server APIOverviewAssistantsThreadsThread RunsCrons (Plus tier)Stateless RunsStoreA2AMCPSystemControl Plane APILangGraph CLIRemoteGraphAgent Server environment variablesReleasesAgent Server changelogSelf-hosted changelogRelease versionsOn this pageAuthenticationLangSmith DeploymentAgent Server APIAgent Server API reference for LangSmith DeploymentCopy pageCopy pageThe Agent Server API reference is available within each deployment at the /docs endpoint (e.g. http://localhost:8124/docs).'

In [18]:
from langchain_community.llms import Ollama

llm = Ollama(
    model="gemma2:2b",
)


In [19]:
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains.combine_documents import create_stuff_documents_chain

prompt = ChatPromptTemplate.from_messages([
    """
    Answer the question based only on the provided context
    <context>
    {context}
    </context>
    """
])

documents_chain = create_stuff_documents_chain(llm, prompt)
documents_chain

RunnableBinding(bound=RunnableBinding(bound=RunnableAssign(mapper={
  context: RunnableLambda(format_docs)
}), config={'run_name': 'format_inputs'})
| ChatPromptTemplate(input_variables=['context'], messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], template='\n    Answer the question based only on the provided context\n    <context>\n    {context}\n    </context>\n    '))])
| Ollama(model='gemma2:2b')
| StrOutputParser(), config={'run_name': 'stuff_documents_chain'})

In [21]:
from langchain_core.documents import Document
documents_chain.invoke({
  "input": "For deployments to LangSmith, authentication is required",
  "context": [Document(page_content="For deployments to LangSmith, authentication is required. Pass the X-Api-Key header with each request to the Agent Server")]
})

'You must include the X-Api-Key header in your requests to the Agent Server for LangSmith deployments. \n'

In [29]:
retriever = vectorstoredb.as_retriever()

from langchain.chains import create_retrieval_chain

retrievel_chain= create_retrieval_chain(retriever,documents_chain)

In [30]:
retrievel_chain

RunnableBinding(bound=RunnableAssign(mapper={
  context: RunnableBinding(bound=RunnableLambda(lambda x: x['input'])
           | VectorStoreRetriever(tags=['FAISS', 'OllamaEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x00000284A2C08C70>), config={'run_name': 'retrieve_documents'})
})
| RunnableAssign(mapper={
    answer: RunnableBinding(bound=RunnableBinding(bound=RunnableAssign(mapper={
              context: RunnableLambda(format_docs)
            }), config={'run_name': 'format_inputs'})
            | ChatPromptTemplate(input_variables=['context'], messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], template='\n    Answer the question based only on the provided context\n    <context>\n    {context}\n    </context>\n    '))])
            | Ollama(model='gemma2:2b')
            | StrOutputParser(), config={'run_name': 'stuff_documents_chain'})
  }), config={'run_name': 'retrieval_chain'})

In [31]:
response = retrievel_chain.invoke({"input":"For deployments to LangSmith, authentication is required"})

In [32]:
response['answer']

"The provided text explains that authentication is required for deployments to LangSmith and details how to do so.  \n\nHere's a summary of the key points regarding authentication:\n\n* **X-Api-Key header:** This header must be included in each request to the Agent Server with the correct API key.\n* **Valid API key:** The API key must belong to the organization where the Agent server is deployed. \n* **Example curl command:**  The provided example shows how to make a POST request using the `curl` tool with an X-Api-Key header and JSON data to access the `/assistants/search` endpoint. \n\n\n\nLet me know if you'd like any other information or clarification on this! \n"